In [1]:
import pandas as pd
import numpy as np
import datetime
import copy

from IJCAI2017_TOOL import *

TEST = pd.read_csv('./TEST_SELLS.csv')

In [5]:
TEST.head()

,SHOP_ID,SA00,SA01,SA02,SA03,SA04,SA05,SA06,SA07,SA08,...,SA15,SA16,SA17,SA18,SA19,SA20,TRN_STA,TRN_END,TST_STA,TST_END
0,1,254.0,239.0,228.0,243.584963,264.0,264.000000,206.584963,244.0,177.000000,...,172.0,226.0,257.0,251.0,280.584963,180.0,2016-10-11,2016-10-31,2016-11-01,2016-11-14
1,2,83.0,88.0,87.0,95.000000,133.0,134.169925,106.000000,90.0,108.754887,...,97.0,83.0,96.0,96.0,115.000000,80.0,2016-10-11,2016-10-31,2016-11-01,2016-11-14
2,3,68.0,63.0,56.0,71.000000,119.0,97.000000,75.000000,59.0,71.000000,...,61.0,68.0,103.0,102.0,104.000000,67.0,2016-10-11,2016-10-31,2016-11-01,2016-11-14
3,4,86.0,113.0,71.0,121.000000,165.0,200.000000,89.000000,90.0,112.000000,...,104.0,55.0,109.0,215.0,195.000000,115.0,2016-10-11,2016-10-31,2016-11-01,2016-11-14
4,5,223.0,218.0,230.0,195.000000,185.0,211.000000,274.000000,201.0,210.000000,...,350.0,250.0,215.0,203.0,291.000000,239.0,2016-10-11,2016-10-31,2016-11-01,2016-11-14


In [2]:
Cor_array = []
for ind,value in TEST.iterrows():
    tt2 = value[np.arange(1,22)].transpose()
    tt3 = np.asarray(tt2,dtype = np.float32).reshape([7,3],order = 'F')
    tt4 = pd.DataFrame( tt3 )
    tt5 = np.corrcoef(tt3,rowvar=0) #计算三周间的相关度矩阵（3×3的对称矩阵，Aij表示i行与j列的相关系数）
    tt6 = tt5[np.triu_indices(3,k=1)]# np.triu_indices(3,k=1)返回矩阵上三角元素的索引
    Cor_array.append(tt6)

/home/zlq/anaconda2/lib/python2.7/site-packages/numpy/lib/function_base.py:3003: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]


In [3]:
Cor_array

[array([ 0.29905428,  0.7631138 ,  0.74541652]),
 array([ 0.75590513,  0.64631739,  0.78612773]),
 array([ 0.68990894,  0.70574281,  0.81728113]),
 array([ 0.94206725,  0.91434829,  0.95166622]),
 array([ 0.91094228,  0.16635545, -0.09950071]),
 array([-0.60490118,  0.72442241, -0.2909982 ]),
 array([ 0.94280302,  0.74349403,  0.66327322]),
 array([ 0.62902345,  0.51931028,  0.23822307]),
 array([ 0.70439759,  0.61716427,  0.57155046]),
 array([ 0.41676482,  0.20552351,  0.03942747]),
 array([ 0.26992293,  0.25423045,  0.63414651]),
 array([ 0.23712565,  0.63027083,  0.7978716 ]),
 array([-0.30127969,  0.03449706, -0.28107004]),
 array([ 0.60499971, -0.11359685,  0.51395761]),
 array([ 0.75248321,  0.93370287,  0.8558035 ]),
 array([ 0.16139664,  0.23467696,  0.17753882]),
 array([ 0.90507339,  0.69399741,  0.72395075]),
 array([ 0.64672557,  0.59486815,  0.52593623]),
 array([ 0.78430915,  0.88669551,  0.56891331]),
 array([ 0.32639231,  0.2858232 ,  0.60621091]),
 array([ 0.96440016,

In [6]:
Cor_columns = [(lambda x:('Cor'+ str(x).zfill(2))) (x)  for x in range( 3 )]
Cor_columns_in = ['Cor00','Cor01','Cor04']
Cor_array_pd =  pd.DataFrame(Cor_array, columns = Cor_columns_in)
Cor_array_pd.head()

,Cor00,Cor01,Cor04
0,0.299054,0.763114,0.745417
1,0.755905,0.646317,0.786128
2,0.689909,0.705743,0.817281
3,0.942067,0.914348,0.951666
4,0.910942,0.166355,-0.099501


In [7]:
#将三周销量相关度矩阵加入数据集，并按照平均相关系数降序排序
TEST = pd.concat([TEST,Cor_array_pd], axis=1)

TEST['Cor_mean'] = TEST[['Cor00','Cor01','Cor04']].mean(axis = 1)

TEST_cor = TEST.sort_values(by = ['Cor_mean','Cor04'], ascending  = False )

In [8]:
HIGH_LIMIT = 0.7

CRITERIA = 0.75

TEST_cor_candidate = TEST_cor[TEST_cor['Cor_mean']>0.75]#选出平均相关系数大于0.75的行
TEST_cor_candidate['Cor_min'] = TEST_cor_candidate[['Cor_mean','Cor04']].min(axis = 1)#选除平均相关系数、最近相关系数中较小的一个
TEST_cor_candidate['credit'] = (TEST_cor_candidate['Cor_min']-CRITERIA)*HIGH_LIMIT/ (1.0- CRITERIA )#计算融合系数（置信度）
TEST_cor_candidate = TEST_cor_candidate[TEST_cor_candidate['credit']>0].sort_values(by = ['credit'], ascending  = False)
TEST_cor_candidate.index = np.arange( len(TEST_cor_candidate) )

/home/zlq/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/zlq/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [9]:
TEST_cor_candidate.head()

,SHOP_ID,SA00,SA01,SA02,SA03,SA04,SA05,SA06,SA07,SA08,...,TRN_STA,TRN_END,TST_STA,TST_END,Cor00,Cor01,Cor04,Cor_mean,Cor_min,credit
0,1624,349.584963,369.584963,340.754888,405.584963,477.169925,473.754888,361.169925,341.000000,352.584963,...,2016-10-11,2016-10-31,2016-11-01,2016-11-14,0.994110,0.995651,0.997025,0.995595,0.995595,0.687667
1,1709,879.000000,845.169925,824.169925,844.000000,192.000000,172.584963,916.000000,912.584963,843.169925,...,2016-10-11,2016-10-31,2016-11-01,2016-11-14,0.994919,0.995017,0.994101,0.994679,0.994101,0.683484
2,1178,223.000000,239.000000,231.000000,223.000000,79.000000,61.000000,249.000000,227.000000,215.000000,...,2016-10-11,2016-10-31,2016-11-01,2016-11-14,0.987582,0.997394,0.993742,0.992906,0.992906,0.680138
3,1875,776.584963,735.584963,793.584963,753.000000,37.000000,49.000000,808.584963,820.000000,825.000000,...,2016-10-11,2016-10-31,2016-11-01,2016-11-14,0.994321,0.997649,0.992095,0.994688,0.992095,0.677866
4,517,2501.209025,1661.114288,1618.604513,1844.774438,3186.078200,3246.963913,1580.209025,2653.284213,1480.679700,...,2016-10-11,2016-10-31,2016-11-01,2016-11-14,0.990616,0.989900,0.994517,0.991678,0.991678,0.676698


In [10]:
average_list = []
increase_list1 = []
increase_list2 = []
for ind,value in TEST_cor_candidate.iterrows():
    w1 = value[np.arange(1,8)].values#选出第一周的销量
    w2 = value[np.arange(8,15)].values
    w3 = value[np.arange(15,22)].values
    w_average = (w1 +w2 +w3)/3.0 #三周按工作日的平均销量
    average_list.append(w_average)
    #通过过去三周按周统计的销量中位数及平均值，做线性拟合得到销量增量。
    increase_list1.append((0.62*(w3.mean() - w2.mean()) + 0.38*(w2.mean() - w1.mean())))
    increase_list2.append((0.62*(np.median(w3) - np.median(w2)) + 0.38*(np.median(w2)- np.median(w1))))

In [15]:
average_list_pd = pd.DataFrame(average_list,columns = np.arange(7))
increase_list1_pd = pd.DataFrame(increase_list1,columns = ['increase1'])
increase_list2_pd = pd.DataFrame(increase_list2,columns = ['increase2'])

TEST_cor_candidate = pd.concat([TEST_cor_candidate,average_list_pd,increase_list1_pd,increase_list2_pd ], axis = 1)

TEST_cor_candidate['7mean'] = TEST.iloc[:,1:8].mean(axis = 1)#计算第一周的平均销量
#选出销量增量不大于平均销量25%的样本（更实际）
TEST_cor_candidate = TEST_cor_candidate[np.abs(TEST_cor_candidate['increase1']/TEST_cor_candidate['7mean']) < 0.25]
TEST_cor_candidate = TEST_cor_candidate[np.abs(TEST_cor_candidate['increase2']/TEST_cor_candidate['7mean']) < 0.25]
TEST_cor_candidate['increase'] = (0.5* TEST_cor_candidate['increase2'] + 0.5 * TEST_cor_candidate['increase1'])*1.0

In [16]:
TEST_cor_candidate.head()

,SHOP_ID,SA00,SA01,SA02,SA03,SA04,SA05,SA06,SA07,SA08,...,1,2,3,4,5,6,increase1,increase2,7mean,increase
0,1624,349.584963,369.584963,340.754888,405.584963,477.169925,473.754888,361.169925,341.000000,352.584963,...,358.723308,329.918296,399.861654,462.056642,467.308271,353.641604,1.169310,-5.582677,242.738561,-2.206683
1,1709,879.000000,845.169925,824.169925,844.000000,192.000000,172.584963,916.000000,912.584963,843.169925,...,839.836592,861.056642,825.723308,182.056642,171.528321,908.194988,-0.858458,0.819218,103.738561,-0.019620
2,1178,223.000000,239.000000,231.000000,223.000000,79.000000,61.000000,249.000000,227.000000,215.000000,...,227.000000,235.000000,213.000000,68.000000,54.666667,250.666667,-3.854286,0.680000,78.428571,-1.587143
3,1875,776.584963,735.584963,793.584963,753.000000,37.000000,49.000000,808.584963,820.000000,825.000000,...,777.528321,797.528321,749.666667,48.333333,49.666667,818.723308,9.788030,7.700000,120.714286,8.744015
5,460,258.000000,267.000000,267.000000,311.000000,91.000000,26.584963,270.000000,251.000000,285.584963,...,269.861654,271.000000,299.666667,92.000000,35.723308,268.000000,-0.848571,1.340000,102.428571,0.245714


In [17]:
#预测接下来两周
TEST_cor_candidate[['01','02','03','04','05','06','07' ]]  = TEST_cor_candidate.iloc[:,1:8].sub(-0.9*TEST_cor_candidate['increase'],axis = 0)
TEST_cor_candidate[[ '08','09','10','11','12','13','14'  ]]  = TEST_cor_candidate.iloc[:,1:8].sub(-1.7*TEST_cor_candidate['increase'],axis = 0)

In [18]:
colo_list = ['SHOP_ID','01','02','03','04','05','06','07', '08','09','10','11','12','13','14' ,'credit']
TEST_cor_candidate = TEST_cor_candidate[colo_list]

TEST_cor_candidate.to_csv('cor_model.csv',index = False)

In [19]:
TEST_cor_candidate

,SHOP_ID,01,02,03,04,05,06,07,08,09,10,11,12,13,14,credit
0,1624,347.598948,367.598948,338.768873,403.598948,475.183910,471.768873,359.183910,345.833601,365.833601,337.003526,401.833601,473.418564,470.003526,357.418564,0.687667
1,1709,878.982342,845.152267,824.152267,843.982342,191.982342,172.567305,915.982342,878.966646,845.136571,824.136571,843.966646,191.966646,172.551609,915.966646,0.683484
2,1178,221.571571,237.571571,229.571571,221.571571,77.571571,59.571571,247.571571,220.301857,236.301857,228.301857,220.301857,76.301857,58.301857,246.301857,0.680138
3,1875,784.454576,743.454576,801.454576,760.869614,44.869614,56.869614,816.454576,791.449788,750.449788,808.449788,767.864826,51.864826,63.864826,823.449788,0.677866
5,460,258.221143,267.221143,267.221143,311.221143,91.221143,26.806105,270.221143,258.417714,267.417714,267.417714,311.417714,91.417714,27.002677,270.417714,0.673827
6,34,372.771286,350.771286,322.771286,458.771286,543.771286,458.771286,375.771286,377.012429,355.012429,327.012429,463.012429,548.012429,463.012429,380.012429,0.670392
7,1694,32.022777,18.022777,26.022777,27.022777,108.607739,128.022777,26.022777,35.598578,21.598578,29.598578,30.598578,112.183541,131.598578,29.598578,0.669476
8,1867,256.507356,233.922394,235.092319,232.922394,61.922394,41.922394,220.922394,259.993928,237.408966,238.578891,236.408966,65.408966,45.408966,224.408966,0.666791
9,1444,57.066714,61.066714,46.066714,84.066714,154.066714,138.066714,64.066714,60.681571,64.681571,49.681571,87.681571,157.681571,141.681571,67.681571,0.662677
10,1855,225.091783,222.091783,210.091783,234.091783,123.091783,79.091783,220.091783,235.840035,232.840035,220.840035,244.840035,133.840035,89.840035,230.840035,0.659009
